In [1]:
import torch
import torchvision
from matplotlib import pyplot as plt
from torchvision import datasets, transforms, models
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


cuda


In [3]:


trainset_vit = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_vit)

testset_vit = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_vit)

trainset_cnn = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_cnn)

testset_cnn = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_cnn)

trainloader_vit = torch.utils.data.DataLoader(
    trainset_vit, batch_size=128, shuffle=True)

testloader_vit = torch.utils.data.DataLoader(
    testset_vit, batch_size=128, shuffle=False)

trainloader_cnn = torch.utils.data.DataLoader(
    trainset_cnn, batch_size=128, shuffle=True)

testloader_cnn = torch.utils.data.DataLoader(
    testset_cnn, batch_size=128, shuffle=False)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*8*8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [5]:
from torchvision.models import vit_b_16

def get_vit_model(num_classes=10, freeze_backbone=True):

    model_vit = vit_b_16(weights="IMAGENET1K_V1")
    model_vit.heads.head = nn.Linear(
        model_vit.heads.head.in_features, num_classes)

    if freeze_backbone:
        for param in model_vit.parameters():
            param.requires_grad = False
        for param in model_vit.heads.head.parameters():
            param.requires_grad = True
    return model_vit

In [6]:
# def train_and_test(model, train_loader, test_loader,
#                    epochs, lr, name):
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     model.to(device)
#
#     criterion = nn.CrossEntropyLoss()
#     if hasattr(model, "heads"):  # για ViT
#         optimizer = torch.optim.Adam(model.heads.head.parameters(), lr=lr)
#     else:  # για CNN
#         optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#
#
#     print(f"\nTraining {name}")
#
#     for epoch in range(epochs):
#         model.train()
#         running_loss = 0.0
#
#         pbar = tqdm(train_loader,
#                     desc=f"Epoch [{epoch+1}/{epochs}]")
#
#         for images, labels in pbar:
#             images, labels = images.to(device), labels.to(device)
#
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#
#             running_loss += loss.item()
#             pbar.set_postfix(loss=loss.item())
#
#         print(f"Epoch [{epoch+1}/{epochs}] - "
#               f"Loss: {running_loss/len(train_loader):.4f}")
#
#     # ---- TEST ----
#     model.eval()
#     correct, total = 0, 0
#     with torch.no_grad():
#         for images, labels in test_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#
#     acc = 100 * correct / total
#     print(f"{name} Accuracy: {acc:.2f}%")
#     return acc


In [7]:
def train_one_epoch(model, loader, optimizer, criterion, device, epoch):
    model.train()
    running_loss = 0.0
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix(
            loss=loss.item(),
            ac
        )
    return running_loss / len(loader)

In [8]:
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [9]:
def train_model(model, train_loader, test_loader, epochs, lr, name, device):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    if hasattr(model, "heads"):
        optimizer = torch.optim.Adam(model.heads.head.parameters(), lr=lr)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {"train_loss": [], "val_acc": []}

    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)
        val_acc = evaluate(model, test_loader, device)
        history["train_loss"].append(train_loss)
        history["val_acc"].append(val_acc)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.2f}%")
    return history

In [10]:
def plot_history(histories):
    plt.figure(figsize=(12,5))
    for name, history in histories.items():
        plt.plot(history["train_loss"], label=f"{name} Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training Loss Comparison")
    plt.show()

    plt.figure(figsize=(12,5))
    for name, history in histories.items():
        plt.plot(history["val_acc"], label=f"{name} Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.legend()
    plt.title("Validation Accuracy Comparison")
    plt.show()

In [11]:
results = {}

# 1) CNN, 3 epochs, lr=1e-3
cnn_model = SimpleCNN()
history_cnn = train_model(cnn_model, trainloader_cnn, testloader_cnn,
                          epochs=3, lr=1e-3, name="CNN 3ep", device=device)
results["CNN 3ep"] = history_cnn

Epoch 1: 100%|██████████| 391/391 [00:19<00:00, 20.38it/s, loss=1.11] 


Epoch 1/3 - Train Loss: 1.3773, Val Acc: 61.55%


Epoch 2: 100%|██████████| 391/391 [00:15<00:00, 25.68it/s, loss=0.994]


Epoch 2/3 - Train Loss: 0.9773, Val Acc: 67.08%


Epoch 3: 100%|██████████| 391/391 [00:15<00:00, 25.36it/s, loss=0.736]


Epoch 3/3 - Train Loss: 0.8132, Val Acc: 70.32%


In [12]:
# 2) CNN, 5 epochs, lr=1e-3
cnn_model = SimpleCNN()
history_cnn5 = train_model(cnn_model, trainloader_cnn, testloader_cnn,
                           epochs=5, lr=1e-3, name="CNN 5ep", device=device)
results["CNN 5ep"] = history_cnn5

Epoch 1: 100%|██████████| 391/391 [00:15<00:00, 25.09it/s, loss=0.987]


Epoch 1/5 - Train Loss: 1.3926, Val Acc: 59.59%


Epoch 2: 100%|██████████| 391/391 [00:15<00:00, 25.02it/s, loss=0.914]


Epoch 2/5 - Train Loss: 1.0269, Val Acc: 66.21%


Epoch 3: 100%|██████████| 391/391 [00:15<00:00, 24.59it/s, loss=0.839]


Epoch 3/5 - Train Loss: 0.8662, Val Acc: 69.45%


Epoch 4: 100%|██████████| 391/391 [00:15<00:00, 25.98it/s, loss=0.987]


Epoch 4/5 - Train Loss: 0.7448, Val Acc: 69.38%


Epoch 5: 100%|██████████| 391/391 [00:15<00:00, 25.61it/s, loss=0.631]


Epoch 5/5 - Train Loss: 0.6411, Val Acc: 72.13%


In [13]:
# 3) ViT, 3 epochs, lr=1e-3
vit_model = get_vit_model()
history_vit = train_model(vit_model, trainloader_vit, testloader_vit,
                          epochs=3, lr=1e-3, name="ViT 3ep", device=device)
results["ViT 3ep"] = history_vit

Epoch 1:   5%|▌         | 21/391 [01:10<20:44,  3.36s/it, loss=0.643]


KeyboardInterrupt: 

In [ ]:
# 4) ViT, 5 epochs, lr=1e-3
vit_model = get_vit_model()
history_vit5 = train_model(vit_model, trainloader_vit, testloader_vit,
                           epochs=5, lr=1e-3, name="ViT 5ep", device=device)
results["ViT 5ep"] = history_vit5

In [17]:

plot_history(results)



Training CNN (5 epochs)


Epoch [1/5]: 100%|██████████| 391/391 [00:14<00:00, 26.83it/s, loss=1.23] 


Epoch [1/5] - Loss: 1.3963


Epoch [2/5]: 100%|██████████| 391/391 [00:14<00:00, 27.59it/s, loss=0.748]


Epoch [2/5] - Loss: 1.0092


...

Epoch [3/5] - Loss: 0.8578


Epoch [4/5]: 100%|██████████| 391/391 [00:14<00:00, 27.54it/s, loss=0.603]


Epoch [4/5] - Loss: 0.7464


Epoch [5/5]: 100%|██████████| 391/391 [00:14<00:00, 27.56it/s, loss=0.717]


Epoch [5/5] - Loss: 0.6491
CNN (5 epochs) Accuracy: 72.24%
